In [1]:
# step1: (i, (ai1, .., ain))                                // read file and add index i (have row uindex in txt file)
# step2: (i, ((ai1, 1),..,(ain, n))) using a for loop       // add index of column For j in range(n): yield (aij, j)
# step3: (i, vi)    w                                      // add index of vector (use for loop and yield)
# step4: (j, (i, aij)) j: column number            z         // flatmap -> (i, (aij, j))   ==>  change pos of i and j
# step5: (j, ((i, aij), vj))                                // z.join(w) to join step3 and step4
# step6: (j, (i, aijvj))                                    // multiplies vj in
# step7: (i, aijvj)                                         // move out i and remove j
# step8: (i, sum(aijvj) over j))                            // sum over j (reduce)

In [2]:
A = sc.textFile('A')
v = sc.textFile('v')

In [3]:
A = A.map(lambda l: (l.split(' ')))
A = A.map(lambda l: (int(l[0]), tuple(map(float, l[1].split(',')))))

In [4]:
A.collect()

[(0, (1.0, 2.0, 3.0)), (1, (4.0, 5.0, 6.0)), (2, (7.0, 8.0, 9.0))]

In [5]:
def add_col_index(l):
    for idx, val in enumerate(l):
        yield (idx, val)

In [6]:
A = A.map(lambda l: (l[0], tuple(enumerate(l[1]))))

In [7]:
A.collect()

[(0, ((0, 1.0), (1, 2.0), (2, 3.0))),
 (1, ((0, 4.0), (1, 5.0), (2, 6.0))),
 (2, ((0, 7.0), (1, 8.0), (2, 9.0)))]

In [8]:
v = v.map(lambda l: tuple(map(float, l.split(','))))
v = v.map(lambda l: tuple(enumerate(l)))
v = v.flatMap(lambda l: l)

In [9]:
v.collect()

[(0, 10.0), (1, 11.0), (2, 12.0)]

In [10]:
def combine(idx, t):
    for tup in t:
        yield (idx, tup)

In [11]:
flat_A = A.flatMap(lambda l: tuple(combine(l[0], l[1])))
flat_A = flat_A.map(lambda l: tuple((l[1][0], (l[0], l[1][1]))))

In [12]:
flat_A.collect()

[(0, (0, 1.0)),
 (1, (0, 2.0)),
 (2, (0, 3.0)),
 (0, (1, 4.0)),
 (1, (1, 5.0)),
 (2, (1, 6.0)),
 (0, (2, 7.0)),
 (1, (2, 8.0)),
 (2, (2, 9.0))]

In [13]:
joined = flat_A.join(v)

In [14]:
joined.collect()

[(0, ((0, 1.0), 10.0)),
 (0, ((1, 4.0), 10.0)),
 (0, ((2, 7.0), 10.0)),
 (1, ((0, 2.0), 11.0)),
 (1, ((1, 5.0), 11.0)),
 (1, ((2, 8.0), 11.0)),
 (2, ((0, 3.0), 12.0)),
 (2, ((1, 6.0), 12.0)),
 (2, ((2, 9.0), 12.0))]

In [15]:
joined_ = joined.map(lambda l: (l[0], (l[1][0][0], l[1][0][1]*l[1][1])))

In [16]:
joined_.collect()

[(0, (0, 10.0)),
 (0, (1, 40.0)),
 (0, (2, 70.0)),
 (1, (0, 22.0)),
 (1, (1, 55.0)),
 (1, (2, 88.0)),
 (2, (0, 36.0)),
 (2, (1, 72.0)),
 (2, (2, 108.0))]

In [17]:
joined_ = joined_.map(lambda l: (l[1][0], l[1][1]))

In [18]:
joined_.collect()

[(0, 10.0),
 (1, 40.0),
 (2, 70.0),
 (0, 22.0),
 (1, 55.0),
 (2, 88.0),
 (0, 36.0),
 (1, 72.0),
 (2, 108.0)]

In [19]:
sum_ = joined_.reduceByKey(lambda n1, n2: n1+n2)

In [26]:
sum_.collect()

[(0, 68.0), (1, 167.0), (2, 266.0)]

In [36]:
output = sum_.map(lambda l: l[1])

In [37]:
output.collect()

[68.0, 167.0, 266.0]